In [1]:
import pandas as pd 
import numpy as np 

#ML
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.metrics import accuracy_score
from scipy import stats
import matplotlib.pyplot as plt
import os
import random
from sklearn.metrics import r2_score
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from scipy import stats
import matplotlib.pyplot as plt
import os



In [2]:
def confidence_interval(accuracy, n, confidence=0.95):
    std_error = np.sqrt(accuracy * (1 - accuracy) / n)
    z_score = stats.norm.ppf((1 + confidence) / 2)
    margin_of_error = z_score * std_error
    return (accuracy - margin_of_error, accuracy + margin_of_error)


In [3]:

all_simple = pd.DataFrame()
all_complex = pd.DataFrame()

for counter_p_small_world in range(1,10):
    #finding p
    p_small_world = counter_p_small_world*0.1
    p_small_world = np.round(p_small_world,3)
    print(p_small_world)

    #network name set
    network_name = f'smallworld_p0.{counter_p_small_world}'
    
    #load data sets
    df1 = pd.read_csv(f"../result_small_world/unweighted/unweighted_complex_{network_name}_theta_2_prob_0.02_EPH-v3.csv")
    df2 = pd.read_csv(f"../result_small_world/unweighted/unweighted_complex_{network_name}_theta_3_prob_0.02_EPH-v3.csv")
    df3 = pd.read_csv(f"../result_small_world/unweighted/unweighted_complex_{network_name}_theta_4_prob_0.02_EPH-v3.csv")
    df4 = pd.read_csv(f"../result_small_world/unweighted/unweighted_complex_{network_name}_theta_5_prob_0.02_EPH-v3.csv")
    df5 = pd.read_csv(f"../result_small_world/unweighted/unweighted_complex_{network_name}_theta_6_prob_0.02_EPH-v3.csv")
    df6 = pd.read_csv(f"../result_small_world/unweighted/unweighted_complex_{network_name}_theta_7_prob_0.02_EPH-v3.csv")
    df7 = pd.read_csv(f"../result_small_world/unweighted/unweighted_complex_{network_name}_theta_8_prob_0.02_EPH-v3.csv")

    df8 = pd.read_csv(f"../result_small_world/unweighted/unweighted_simple_{network_name}_beta_0.02_EPH-v3.csv")
    df9 = pd.read_csv(f"../result_small_world/unweighted/unweighted_simple_{network_name}_beta_0.03_EPH-v3.csv")
    df10 = pd.read_csv(f"../result_small_world/unweighted/unweighted_simple_{network_name}_beta_0.04_EPH-v3.csv")
    df11 = pd.read_csv(f"../result_small_world/unweighted/unweighted_simple_{network_name}_beta_0.05_EPH-v3.csv")


    df1['p_small_world'] = p_small_world 
    df2['p_small_world'] = p_small_world
    df3['p_small_world'] = p_small_world
    df4['p_small_world'] = p_small_world
    df5['p_small_world'] = p_small_world
    df6['p_small_world'] = p_small_world
    df7['p_small_world'] = p_small_world
    
    complex_c = pd.concat([df1,df2])
    complex_c = pd.concat([complex_c,df3])
    complex_c = pd.concat([complex_c,df4])
    complex_c = pd.concat([complex_c,df5])
    complex_c = pd.concat([complex_c,df6])
    complex_c = pd.concat([complex_c,df7])
    complex_c = complex_c.sample(frac=1,random_state=42)
    all_complex = pd.concat([complex_c,all_complex])


    df8['p_small_world'] = p_small_world 
    df9['p_small_world'] = p_small_world
    df10['p_small_world'] = p_small_world
    df11['p_small_world'] = p_small_world
    simple_c = pd.concat([df8,df9])
    simple_c = pd.concat([simple_c,df10])
    simple_c = pd.concat([simple_c,df11])
    simple_c = simple_c.sample(frac=1,random_state=42)
    all_simple = pd.concat([simple_c,all_simple])


# Lists to store results
p_samll_world_set = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
accuracies_tda = []
ci_lower_tda = []
ci_upper_tda = []
accuracies_prl = []
ci_lower_prl = []
ci_upper_prl = []

for p_small_world in p_samll_world_set:
    # Prepare data
    complex_curr = all_complex[all_complex['p_small_world'] == p_small_world]
    complex_curr = complex_curr[['EPH','corr']]
    complex_curr['complex'] = 1

    simple_curr = all_simple[all_simple['p_small_world'] == p_small_world]
    simple_curr = simple_curr[['EPH','corr']]
    simple_curr['complex'] = 0

    if len(simple_curr) > len(complex_curr):
        simple_curr = simple_curr.sample(len(complex_curr),random_state=1242)
    else:
        complex_curr = complex_curr.sample(len(simple_curr),random_state=1242)

    merged = pd.concat([simple_curr,complex_curr])
    merged = merged.sample(len(merged),random_state=1)

    X_tda = merged['EPH']
    X_prl = merged['corr']
    y = merged['complex']

    print(X_tda)
    # Split data
    X_tda_train, X_tda_test, y_tda_train, y_tda_test = train_test_split(X_tda, y, test_size=0.2, random_state=42)
    X_prl_train, X_prl_test, y_prl_train, y_prl_test = train_test_split(X_prl, y, test_size=0.2, random_state=42)

    # Train and evaluate models
    dt_tda = DecisionTreeClassifier(random_state=42)
    dt_prl = DecisionTreeClassifier(random_state=42)

    dt_tda.fit(X_tda_train.values.reshape(-1, 1), y_tda_train)
    dt_prl.fit(X_prl_train.values.reshape(-1, 1), y_prl_train)

    y_tda_pred = dt_tda.predict(X_tda_test.values.reshape(-1, 1))
    y_prl_pred = dt_prl.predict(X_prl_test.values.reshape(-1, 1))

    accuracy_tda = accuracy_score(y_tda_test, y_tda_pred)
    accuracy_prl = accuracy_score(y_prl_test, y_prl_pred)

    # Calculate confidence intervals
    ci_tda = confidence_interval(accuracy_tda, len(y_tda_test))
    ci_prl = confidence_interval(accuracy_prl, len(y_prl_test))

    # Store results
    accuracies_tda.append(accuracy_tda)
    ci_lower_tda.append(ci_tda[0])
    ci_upper_tda.append(min(1,ci_tda[1]))
    accuracies_prl.append(accuracy_prl)
    ci_lower_prl.append(ci_prl[0])
    ci_upper_prl.append(min(1,ci_prl[1]))



0.1
0.2
0.3
0.4
0.5
0.6
0.7
0.8
0.9
199     7.519144
6       2.065374
107     7.291643
45     11.227321
117     2.363258
         ...    
51      5.533445
183     2.849408
152     1.375904
20      5.916102
37      6.377640
Name: EPH, Length: 1600, dtype: float64
199     7.315565
6       2.539616
107     8.615764
45     10.849840
117     2.638746
         ...    
51      5.439530
183     3.264030
152     1.281086
20      5.693777
37      6.669683
Name: EPH, Length: 1600, dtype: float64
199     7.380746
6       2.226031
107     6.889128
45     10.409475
117     2.714371
         ...    
51      5.127066
183     3.579222
152     1.411813
20      5.137912
37      7.008658
Name: EPH, Length: 1600, dtype: float64
199    6.761152
6      2.946939
107    6.656336
45     9.774550
117    2.596298
         ...   
51     5.529219
183    3.869196
152    1.248626
20     5.607181
37     7.143638
Name: EPH, Length: 1600, dtype: float64
199    7.007705
6      2.588929
107    7.632839
45     8.046766
117

In [4]:
# Create the plot
plt.figure(figsize=(10, 7), dpi=1200)

plt.plot(p_samll_world_set, accuracies_tda, marker='o', label='EPH')
plt.fill_between(p_samll_world_set, ci_lower_tda, ci_upper_tda, alpha=0.2)

plt.plot(p_samll_world_set, accuracies_prl, marker='s', label='corr(order,deg)')
plt.fill_between(p_samll_world_set, ci_lower_prl, ci_upper_prl, alpha=0.2)

# Set x-axis to show only integer values
plt.xticks(p_samll_world_set)


plt.xlabel('Rewiring Probability',fontsize=15)
plt.ylabel('Accuracy',fontsize=15)
plt.legend(loc='lower right',fontsize=15)

# plt.ylim(0.90,1.01)

plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

# plt.grid(True)

plt.tight_layout()

# Save the plot
plt.show()
plt.savefig(f'../plot-april-final-version/small_world_result.pdf', format='pdf', dpi=1200, bbox_inches='tight')
# plt.close()  # Close the plot to free up memory


<Figure size 640x480 with 0 Axes>